In [1]:
import tkinter as tk
from tkinter import messagebox
import time
import random
from collections import deque

CELL_SIZE = 15  
GRID_WIDTH = 15
GRID_HEIGHT = 15

EMPTY = 0
WALL = 1
START = 2
END = 3
VISITED = 4
PATH = 5

COLORS = {
    EMPTY: "white",
    WALL: "black",
    START: "green",
    END: "red",
    VISITED: "lightblue",
    PATH: "yellow",
}

class MazeSolverGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Maze Solver (BFS / DFS)")
        
       
        self.canvas = tk.Canvas(root, width=GRID_WIDTH*CELL_SIZE, height=GRID_HEIGHT*CELL_SIZE)
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        self.grid = [[EMPTY for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
        self.start = None
        self.end = None
        self.is_solving = False 
        
        
        self.canvas.bind("<Button-1>", self.left_click)
        self.canvas.bind("<Configure>", self.on_resize)  
        
        
        button_frame = tk.Frame(root)
        button_frame.pack()
        
        tk.Button(button_frame, text="Solve with BFS", command=self.solve_bfs).pack(side=tk.LEFT)
        tk.Button(button_frame, text="Solve with DFS", command=self.solve_dfs).pack(side=tk.LEFT)
        tk.Button(button_frame, text="New Game", command=self.new_game).pack(side=tk.LEFT)
        
        self.cell_size = CELL_SIZE  
        self.offset_x = 0  
        self.offset_y = 0  
        self.draw_grid()
    
    def draw_grid(self):
        self.canvas.delete("all")
        for y in range(GRID_HEIGHT):
            for x in range(GRID_WIDTH):
                color = COLORS[self.grid[y][x]]
                self.canvas.create_rectangle(
                    self.offset_x + x * self.cell_size,
                    self.offset_y + y * self.cell_size,
                    self.offset_x + (x + 1) * self.cell_size,
                    self.offset_y + (y + 1) * self.cell_size,
                    fill=color, outline="gray"
                )

    def on_resize(self, event):
        
        canvas_width = event.width
        canvas_height = event.height
        self.cell_size = min(canvas_width / GRID_WIDTH, canvas_height / GRID_HEIGHT)
        
        
        grid_width = self.cell_size * GRID_WIDTH
        grid_height = self.cell_size * GRID_HEIGHT
        self.offset_x = (canvas_width - grid_width) / 2
        self.offset_y = (canvas_height - grid_height) / 2 
        
        self.draw_grid()

    def left_click(self, event):
        if self.is_solving:
            return
        
        x = int((event.x - self.offset_x) // self.cell_size)
        y = int((event.y - self.offset_y) // self.cell_size)

        if not (0 <= x < GRID_WIDTH and 0 <= y < GRID_HEIGHT):
            return

        if self.grid[y][x] == WALL:
            return

        if not self.start:
            self.start = (y, x)
            self.grid[y][x] = START
        elif not self.end and (y, x) != self.start:
            self.end = (y, x)
            self.grid[y][x] = END
        self.draw_grid()

    def new_game(self):
        self.is_solving = False 
        self.grid = [[EMPTY for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
        self.start = None
        self.end = None
        for y in range(GRID_HEIGHT):
            for x in range(GRID_WIDTH):
                if random.random() < 0.3:
                    self.grid[y][x] = WALL
        self.draw_grid()

    def solve_bfs(self):
        if self.is_solving:
            self.is_solving = False  
            self.reset_path()
        if not self.start or not self.end:
            messagebox.showwarning("Missing Points", "Set both start and end points!")
            return
        self.reset_path()
        self.is_solving = True
        self.run_bfs()

    def solve_dfs(self):
        if self.is_solving:
            self.is_solving = False 
            self.reset_path()
        if not self.start or not self.end:
            messagebox.showwarning("Missing Points", "Set both start and end points!")
            return
        self.reset_path()
        self.is_solving = True
        self.run_dfs()

    def run_bfs(self):
        visited = set()
        parent = {}
        queue = deque([self.start])  

        while queue and self.is_solving:
            current = queue.popleft() 

            if current == self.end:
                self.reconstruct_path(parent)
                self.is_solving = False
                return

            y, x = current
            if current != self.start:
                self.grid[y][x] = VISITED
                self.draw_grid()
                self.root.update()
                time.sleep(0.01)

            visited.add(current)

            for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ny, nx = y + dy, x + dx
                neighbor = (ny, nx)
                if 0 <= ny < GRID_HEIGHT and 0 <= nx < GRID_WIDTH:
                    if self.grid[ny][nx] not in [WALL, START] and neighbor not in visited and neighbor not in parent:
                        parent[neighbor] = current
                        queue.append(neighbor)  

        if self.is_solving:
            messagebox.showinfo("Maze Solver", "No path found!")
            self.is_solving = False

    def run_dfs(self):
        visited = set()
        parent = {}
        stack = [self.start]

        while stack and self.is_solving:
            current = stack.pop()

            if current == self.end:
                self.reconstruct_path(parent)
                self.is_solving = False
                return

            y, x = current
            if current != self.start:
                self.grid[y][x] = VISITED
                self.draw_grid()
                self.root.update()
                time.sleep(0.01)

            visited.add(current)

            for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ny, nx = y + dy, x + dx
                neighbor = (ny, nx)
                if 0 <= ny < GRID_HEIGHT and 0 <= nx < GRID_WIDTH:
                    if self.grid[ny][nx] not in [WALL, START] and neighbor not in visited and neighbor not in parent:
                        parent[neighbor] = current
                        stack.append(neighbor)

        if self.is_solving:
            messagebox.showinfo("Maze Solver", "No path found!")
            self.is_solving = False

    def reconstruct_path(self, parent):
        current = self.end
        while current != self.start and self.is_solving:
            y, x = current
            if self.grid[y][x] != END:
                self.grid[y][x] = PATH
            current = parent[current]
            self.draw_grid()
            self.root.update()
            time.sleep(0.03)

    def reset_path(self):
        for y in range(GRID_HEIGHT):
            for x in range(GRID_WIDTH):
                if self.grid[y][x] in [VISITED, PATH]:
                    self.grid[y][x] = EMPTY
        self.draw_grid()

if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("400x400")  
    app = MazeSolverGUI(root)
    root.mainloop()